## clinical variable retrieval codebase.
each querey is referencing a sql querey linked in my github for ALL patients in the database, then generating a dataframe, then paring that dataframe down to only the patients/icustay_id in our cohort. 

In [407]:
#7-15-18
#the final version of this notebook seeks to accomplish a few tasks:
#organize and standardize all sql code so that i am running a .sql file rather than typing sql code into jupyter
#change all filepaths to match the github linked directory to ensure better version control
#extract all of the structured clinical variables we need for our project


##1/28/19: updated final version to be more generalizable and adjustable for the datewindow. 
##could be cleaned up as some variabls are time-windowed in line and some are time windowed at the end, but all is accounted for and cleaned up and optimized.

time: 2.1 ms


In [1]:
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import collections
import asyncio
import getpass
import re
from datetime import datetime as dt
import os,sys,re
import urllib3
import prettytable
from collections import Counter
import seaborn as sns
import random

from sklearn.externals.joblib import Memory
memory = Memory(cachedir='/tmp', verbose=0)
#@memory.cache above any def fxn.

%matplotlib inline
plt.style.use('ggplot')

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

%load_ext autotime

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: The 'cachedir' parameter has been deprecated in version 0.12 and will be removed in version 0.14.
You provided "cachedir='/tmp'", use "location='/tmp'" instead.


In [2]:
from pathlib import Path
os.chdir('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling') #use to change working directory
wd= os.getcwd() #'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling'


time: 2.11 ms


In [109]:
#patients of interest from rotation_cohort_generation

final_pt_df2 = pd.read_csv('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/04042019_final_pt_df2.csv', index_col=0)
date='04042019'

patients= list(final_pt_df2['subject_id'].unique())
hadm_id= list(final_pt_df2['hadm_id'].unique())
icustay_id= list(final_pt_df2['icustay_id'].unique())
icustay_id= [int(x) for x in icustay_id]

time: 111 ms


## Access MIMIC database and convert it to dataframe in Pandas

In [41]:
import psycopg2
conn = psycopg2.connect(dbname="mimic", user="student", host="luolabwkstn01.fsm.northwestern.edu", port="5432",
                        password=getpass.getpass(prompt='Password:'.format()))
cur=conn.cursor()

query_schema = 'SET search_path to ' + "mimiciii" + ';'

Password: ·····


time: 2.38 s


In [5]:
#input the sql_exe_show object and get dataframe for only patients in patient list out. 
def sql_exe_show(sql_sentence):
    cur.execute(sql_sentence)
    rows = cur.fetchall()
    col = []
    for i in range(len(cur.description)):
        col.append(cur.description[i][0])
    table = pd.DataFrame(rows,columns=col)
    return table

def sql_to_df_icu(sql_exe_show_obj):
    sql_exe_show_df= pd.DataFrame(data=sql_exe_show_obj)
    sql_exe_show_df=sql_exe_show_df[sql_exe_show_df['icustay_id'].isin(icustay_id)]
    return sql_exe_show_df

def sql_to_df_patients(sql_exe_show_obj):
    sql_exe_show_df= pd.DataFrame(data=sql_exe_show_obj)
    sql_exe_show_df=sql_exe_show_df[sql_exe_show_df['subject_id'].isin(patients)]
    return sql_exe_show_df

def sql_to_df_hadm(sql_exe_show_obj):
    sql_exe_show_df= pd.DataFrame(data=sql_exe_show_obj)
    sql_exe_show_df=sql_exe_show_df[sql_exe_show_df['hadm_id'].isin(hadm_id)]
    return sql_exe_show_df

def clinvar_fxn(var_name, path):
    f= open(path, 'r')
    var = f.read()
    cur.execute('rollback')
    cur.execute(var)
    try:
        df= sql_to_df_icu(sql_exe_show('select * from %s;' %(var_name)))
    except KeyError or NameError:
        try:      
            df= sql_to_df_hadm(sql_exe_show('select * from %s;' %(var_name)))
        except KeyError or NameError:
            df= sql_to_df_patients(sql_exe_show('select * from %s;' %(var_name)))

    print(df.shape)
    return(df)


time: 49.5 ms


# extracting clinical data for our patients

In [35]:
lower_window=0
upper_window=3

time: 851 µs


# demographics

In [122]:
#gender & race

pt_info_sql = query_schema + """
SELECT SUBJECT_ID, INSURANCE, LANGUAGE, RELIGION, MARITAL_STATUS, ETHNICITY
from mimiciii.admissions
ORDER BY subject_id DESC
"""

pt_info_df=pd.read_sql_query(pt_info_sql,conn) #361711 patients with sterile culture -> 374643 with addn of bal and broncho... 7/16/18


time: 427 ms


In [123]:
ethnicity_df=(pt_info_df.loc[
    pt_info_df.loc[:,"subject_id"].isin(
        final_pt_df2['subject_id'].tolist()),:]).drop_duplicates(['subject_id','ethnicity'])
ethnicity_df= ethnicity_df[['subject_id','ethnicity']].sort_values('ethnicity', ascending=False).groupby('subject_id', as_index=False).first()


time: 30 ms


In [124]:
ethnicity_df

,subject_id,ethnicity
0,9,UNKNOWN/NOT SPECIFIED
1,19,WHITE
2,25,WHITE
3,26,UNKNOWN/NOT SPECIFIED
4,33,UNKNOWN/NOT SPECIFIED
5,61,WHITE
6,62,PATIENT DECLINED TO ANSWER
7,68,BLACK/AFRICAN AMERICAN
8,73,WHITE
9,80,WHITE


time: 18.8 ms


In [125]:
#gender
pt_info_sql = query_schema + """
SELECT SUBJECT_ID, GENDER
from mimiciii.patients
ORDER BY subject_id DESC
"""
#admissions
# 

pt_info_df=pd.read_sql_query(pt_info_sql,conn) #361711 patients with sterile culture -> 374643 with addn of bal and broncho... 7/16/18


time: 117 ms


In [126]:
#combining gender, race
pt_info_df=(pt_info_df.loc[
    pt_info_df.loc[:,"subject_id"].isin(
        final_pt_df2['subject_id'].tolist()),:]).drop_duplicates(['subject_id','gender'])
pt_info_df= pd.merge(ethnicity_df, pt_info_df)

time: 13.4 ms


In [128]:
#age
age_df= pd.read_csv(Path(wd+'/data/processed/22112018_pt_age.csv'))
age_df=(age_df.loc[
    age_df.loc[:,"hadm_id"].isin(
        hadm_id),:])

#combining age, gender and race. 
age_df=pd.merge(age_df, final_pt_df2[['hadm_id','icustay_id','t_0']])
age_df=pd.merge(age_df[['subject_id','icustay_id','first_admit_age','t_0']],pt_info_df)

age_df[age_df['first_admit_age']>89]['first_admit_age']=90

time: 285 ms


/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [129]:
age_df=pd.melt(age_df, id_vars=['icustay_id','subject_id','t_0'])
age_df=age_df.rename(index=str, columns={'variable':'label'})

time: 72 ms


In [130]:
age_df['delta']=pd.to_timedelta('0 days')
age_df['uom']="N/A"
age_df.loc[age_df.loc[:,'label']=='first_admit_age','uom']='years'

time: 38.2 ms


In [131]:
age_df= age_df.loc[age_df.loc[:,"icustay_id"].isin(icustay_id),:]
age_df['label'].unique()

array(['first_admit_age', 'ethnicity', 'gender'], dtype=object)

time: 19 ms


In [132]:
###using regular expressions to reduce the # of ethinicities
age_df.loc[(age_df.loc[:,"label"]=='ethnicity') & (age_df.loc[:,"value"].str.contains(r'.*?BLACK')),'value']="black"
age_df.loc[(age_df.loc[:,"label"]=='ethnicity') & (age_df.loc[:,"value"].str.contains(r'.*?HISPANIC|PORTUGUESE')),'value']="hispanic"
age_df.loc[(age_df.loc[:,"label"]=='ethnicity') & (age_df.loc[:,"value"].str.contains(r'.*?WHITE')),'value']="white/nonhispanic"
age_df.loc[(age_df.loc[:,"label"]=='ethnicity') & (age_df.loc[:,"value"].str.contains(r'.*?ASIAN')),'value']='asian'
age_df.loc[(age_df.loc[:,"label"]=='ethnicity') & (age_df.loc[:,"value"].str.contains(r'(UNKNOWN|MULTI|UNABLE|DECLINE|OTHER)')),'value']='unknown/other'
age_df.loc[(age_df.loc[:,"label"]=='ethnicity') & (age_df.loc[:,"value"].str.contains(r'[AZ]+')),'value']="unknown/other" #lumping all other low n values into other


time: 422 ms


/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  import sys


In [135]:
age_df.loc[age_df.loc[:,"label"]=='ethnicity','value'].value_counts()

white/nonhispanic    10687
black                 1534
unknown/other         1530
hispanic               527
asian                  390
Name: value, dtype: int64

time: 9.09 ms


In [93]:
# pd.DataFrame(age_df).to_csv(Path(
#     wd+'/data/processed/%s_pt_info_prepped.csv' %(date)))

time: 801 ms


## Echodata in Noteevents- sense removed

In [6]:
# echodata_df= clinvar_fxn(
#     'echodata',
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/echodata.sql' 
# )

(7450, 18)
time: 34.8 s


## Weight

In [9]:
# -- This query extracts weights for adult ICU patients on their first ICU day.
# -- It does *not* use any information after the first ICU day, as weight is
# -- sometimes used to monitor fluid balance.

weightfirstday_df= clinvar_fxn(
    'weightfirstday',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/weightfirstday.sql' 
)
weightfirstday_df['uom']='kg'

(14668, 6)
time: 1.97 s


## Height

In [10]:
# -- This query extracts heights for adult ICU patients.
# -- It uses all information from the patient's first ICU day.
# -- This is done for consistency with other queries - it's not necessarily needed.
# -- Height is unlikely to change throughout a patient's stay.
heightfirstday_df= clinvar_fxn(
    'heightfirstday',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/heightfirstday.sql' 
)

heightfirstday_df['uom']='cm'

(14668, 4)
time: 1.25 s


# Labs

In [11]:
# -- This query pivots lab values for all patients, then filtered to those in my cohort.
# -- Have already confirmed that the unit of measurement is always the same: null or the correct unit

labs_all_nosummary_df= clinvar_fxn(
    'labs_all_nosummary',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/no_summary/labs_all_nosummary.sql' 
)

(3267301, 6)
time: 2min 23s


In [12]:
#importing unit of mesurements:
def uom_sql_import(file_path):
    if isinstance(file_path, str)== True:
        f = open(Path(file_path), 'r')
    else:
        f = open(Path(str(file_path)), 'r')

    SQL = open(file_path,'r').read()
    SQL_df= pd.read_sql_query(SQL,conn)   
    return(SQL_df)

time: 8.02 ms


In [13]:
lab_uom= uom_sql_import(Path(wd+'/data/raw/clinical_var_sql/unit_of_mesurement/labs_uom.sql'))
labs_all_nosummary_df = pd.merge(labs_all_nosummary_df, lab_uom, left_on='label', right_on='label')

time: 23.1 s


In [14]:
wd

'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling'

time: 2.59 ms


In [15]:
labs_all_nosummary_df[labs_all_nosummary_df['label']=='LYMPHO%']#.value_counts()
#labs_all_nosummary_df['label'].value_counts()

labs_all_nosummary_df[labs_all_nosummary_df['label']=='WBC']['valuenum'].describe()
#labs_all_nosummary_df[labs_all_nosummary_df['label']=='PLATELET']['valuenum'].describe()

count    192280.000000
mean         11.847600
std           8.971701
min           0.090000
25%           7.300000
50%          10.300000
75%          14.400000
max         471.700000
Name: valuenum, dtype: float64

time: 1.76 s


## Vital Sign

In [76]:
##-- This query pivots the vital signs for the first 24 hours of a patient's stay
##-- Vital signs include heart rate, blood pressure, respiration rate, and temperature

vitals_all_nosummary_df= clinvar_fxn(
    'vitals_all_nosummary',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/no_summary/vitals_all_nosummary.sql' 
)

(14609367, 7)
time: 16min 25s


In [80]:
vitals_all_nosummary_df.head()

,subject_id,hadm_id,icustay_id,charttime,valueuom,vitalid,valuenum
3,9,150750,220597,2149-11-11 00:00:00,mmHg,MeanArtPress,81.0
4,9,150750,220597,2149-11-11 19:00:00,mmHg,MeanArtPress,72.0
5,9,150750,220597,2149-11-11 19:45:00,mmHg,MeanArtPress,76.0
6,9,150750,220597,2149-11-11 20:00:00,mmHg,MeanArtPress,99.0
17,25,129635,203487,2160-11-02 15:00:00,mmHg,MeanArtPress,82.0


time: 11.5 ms


## Glasgow Coma Scale

In [18]:
gcsall_df= clinvar_fxn(
    'gcsall',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/gcsall.sql' 
)

(14668, 8)
time: 1min 41s


In [19]:
#modified on 8/28/18 to have the days annotation.
##--8/28/18: added in epoch as days, in order to help determine btwn t_0 and 72 hour for pts.
gcsall_days_df= clinvar_fxn(
    'gcsall_days',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/gcsall_days.sql' 
)

(14668, 9)
time: 54.5 s


In [23]:
gcsall_days_df.head()

,subject_id,hadm_id,icustay_id,day,mingcs,gcsmotor,gcsverbal,gcseyes,endotrachflag
7,28448,177527,200012,1.0,15.0,6.0,5.0,4.0,0.0
8,9514,127229,200014,2.0,13.0,6.0,3.0,4.0,0.0
12,21789,112486,200019,2.0,6.0,4.0,1.0,1.0,0.0
20,14987,165426,200030,6.0,11.0,5.0,4.0,2.0,0.0
22,56369,198650,200033,12.0,3.0,1.0,1.0,1.0,0.0


time: 47.7 ms


In [24]:
#adding in icu_admit time and filtereing time_var to time window.
def gcs_72(gcsall_days_df,final_pt_df2, time_var='t_0', lower_window=0, upper_window=3):
    
    ##merging gcsall_days_df with final_pt df in order to append on icustay_id, the time var, and ICU_admit
    gcsall_days_df_merge= pd.merge(
    gcsall_days_df,
    final_pt_df2[['icustay_id','ICU_admit',time_var]],
    left_on='icustay_id',
    right_on='icustay_id')

    gcsall_days_df_merge['day'] = gcsall_days_df_merge['day']-1 #putting the epoch days so that 0 = the first day
    
    #approximating the charttime of the time associated with each gcs score
    gcsall_days_df_merge['approx_charttime']=pd.to_timedelta((gcsall_days_df_merge['day'])*24, unit='h') + pd.to_datetime(gcsall_days_df_merge['ICU_admit'])
    
    # day # + ICU_admission day.
    gcsall_days_df_merge['admit_plus_day']= (
        pd.to_datetime(gcsall_days_df_merge['ICU_admit'])
        + pd.to_timedelta(gcsall_days_df_merge['day'], unit='D')
    )
    
    #difference between the admission+epoch day - time_var.
    gcsall_days_df_merge['delta']= (
        pd.to_datetime(gcsall_days_df_merge['admit_plus_day']) - pd.to_datetime(gcsall_days_df_merge[time_var])
    )
    
    #filtering day windows
    gcsall_days_df_merge_72= (
        gcsall_days_df_merge.loc[gcsall_days_df_merge.loc[:,'delta']>= pd.Timedelta(days=lower_window),:])
    gcsall_days_df_merge_72= (
        gcsall_days_df_merge_72.loc[gcsall_days_df_merge_72.loc[:,'delta']<= pd.Timedelta(days=upper_window),:])
    return(gcsall_days_df_merge_72)

time: 22.7 ms


In [27]:
gcs72_df = gcs_72(gcsall_days_df,final_pt_df2, time_var='first_pos_else_neg_ssc', lower_window=lower_window,upper_window=upper_window )

gcs72_df['uom']='GCS_score' #adding in uom


time: 170 ms


In [28]:
# date= '27082018'

pd.DataFrame(gcs72_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_gcs72_df.csv' %(date))

time: 234 ms


## Renal replacement therapy

the sql code for this was not equipped to join all of the charttimes together. so i decided to do it in python below.
the rrt_all_df code above was only a 1 or 0 if patient had RRT during their entire icu stay. 

- step 1: run all sql codes
- 2: filter on only the t_0 to t_72 hour rows
- 3: filter on the 1223 patients who have a positive value
- 4: get the earliest incidence of rrt for each 1223 patients. 

In [29]:
#mv_ce
f = open('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/no_summary/rtt_mv_ce.sql', 'r')
rrtSQL_mv_ce = f.read()
rrtSQL_mv_ce_sql = query_schema + rrtSQL_mv_ce.format(tuple(patients))
rrtSQL_mv_ce_df=pd.read_sql_query(rrtSQL_mv_ce_sql,conn)            

time: 11.2 s


In [30]:
#cv
f = open('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/no_summary/rtt_cv.sql', 'r')
rrtSQL_cv = f.read()
rrtSQL_cv_sql = query_schema + rrtSQL_cv.format(tuple(patients))
rrtSQL_cv_df=pd.read_sql_query(rrtSQL_cv_sql,conn)          

time: 14.4 s


In [31]:
#mv_ie
f = open('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/no_summary/rtt_mv_ie.sql', 'r')
rrtSQL_mv_ie = f.read()
rrtSQL_mv_ie_sql = query_schema + rrtSQL_mv_ie.format(tuple(patients))
rrtSQL_mv_ie_df=pd.read_sql_query(rrtSQL_mv_ie_sql,conn)      
rrtSQL_mv_ie_df['charttime']= rrtSQL_mv_ie_df['starttime']
rrtSQL_mv_ie_df=rrtSQL_mv_ie_df.drop('starttime', axis=1)

time: 3.34 s


In [32]:
#mv_de
f = open('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/no_summary/rtt_mv_de.sql', 'r')
rrtSQL_mv_de = f.read()
rrtSQL_mv_de_sql = query_schema + rrtSQL_mv_de.format(tuple(patients))
rrtSQL_mv_de_df=pd.read_sql_query(rrtSQL_mv_de_sql,conn)          

time: 1.49 s


In [33]:
#mv_pe
f = open('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/no_summary/rtt_mv_pe.sql', 'r')
rrtSQL_mv_pe = f.read()
rrtSQL_mv_pe_sql = query_schema + rrtSQL_mv_pe.format(tuple(patients))
rrtSQL_mv_pe_df=pd.read_sql_query(rrtSQL_mv_pe_sql,conn)          
rrtSQL_mv_pe_df['charttime']= rrtSQL_mv_pe_df['starttime']
rrtSQL_mv_pe_df=rrtSQL_mv_pe_df.drop('starttime', axis=1)

time: 240 ms


In [34]:
def hour_72_window_rrt(df, final_pt_df2,timecol='charttime',time_var='t_0', lower_window=0, upper_window=3 ):
    ##modified to make more generalizable to easily accomidate PA cohort but default to my origional cohort.
    ##filters rrt to within timewindow  between timecol- time_var

    df= pd.merge(final_pt_df2[['icustay_id',time_var]], df, left_on= 'icustay_id', right_on = 'icustay_id', how='left') #n=240317
    df['delta']= pd.to_datetime(df[timecol]) - pd.to_datetime(df[time_var])
    df_after_t0= df.loc[df.loc[:,'delta']>= pd.Timedelta(days=lower_window),:]
    df_after_t0= df_after_t0.loc[df_after_t0.loc[:,'delta']<= pd.Timedelta(days=upper_window),:] 
    #df_after_t0= df_after_t0.loc[df_after_t0.loc[:,'rrt']==1,:].groupby('icustay_id')['charttime'].min()

    return(pd.DataFrame(df_after_t0))#.reset_index())

time: 8.41 ms


In [36]:
rrtSQL_mv_ce_pt =hour_72_window_rrt(rrtSQL_mv_ce_df, final_pt_df2, 'charttime',time_var='first_pos_else_neg_ssc', lower_window=lower_window,upper_window=upper_window)
rrtSQL_cv_pt =hour_72_window_rrt(rrtSQL_cv_df, final_pt_df2, 'charttime',time_var='first_pos_else_neg_ssc', lower_window=lower_window,upper_window=upper_window)
rrtSQL_mv_ie_pt =hour_72_window_rrt(rrtSQL_mv_ie_df, final_pt_df2, 'charttime',time_var='first_pos_else_neg_ssc', lower_window=lower_window,upper_window=upper_window)
rrtSQL_mv_de_pt =hour_72_window_rrt(rrtSQL_mv_de_df, final_pt_df2, 'charttime',time_var='first_pos_else_neg_ssc', lower_window=lower_window,upper_window=upper_window)
rrtSQL_mv_pe_pt =hour_72_window_rrt(rrtSQL_mv_pe_df, final_pt_df2, 'charttime',time_var='first_pos_else_neg_ssc', lower_window=lower_window,upper_window=upper_window)

time: 756 ms


In [37]:
def rrt_merging(rrtSQL_mv_ce_pt, rrtSQL_cv_pt, rrtSQL_mv_ie_pt, rrtSQL_mv_de_pt, rrtSQL_mv_pe_pt, timecol='charttime',time_var='t_0'):
    ###returns an aggregate y/n of if patient had positive rrt within timewindow. 

    rrt_merged_pt= pd.concat([rrtSQL_mv_ce_pt, rrtSQL_cv_pt, rrtSQL_mv_ie_pt, rrtSQL_mv_de_pt, rrtSQL_mv_pe_pt])

    #making a 1 if has positive rrt within timewindow:
    rrt_merged_pt= pd.DataFrame(rrt_merged_pt.loc[rrt_merged_pt.loc[:,'rrt']==1,:].groupby('icustay_id')[timecol].min().reset_index())
    rrt_merged_pt['rrt']=1

    rrt_merged_allpt_df= pd.merge(final_pt_df2[['icustay_id',time_var]], rrt_merged_pt, left_on= 'icustay_id', right_on = 'icustay_id', how='left') #n=240317
    rrt_merged_allpt_df=rrt_merged_allpt_df.rename(index=str, columns={timecol:"first_charttime"})

    rrt_merged_allpt_df['uom']='category' #adding a uom category
    rrt_merged_allpt_df.loc[rrt_merged_allpt_df.loc[:,'rrt'].isnull(),'rrt']='0'
    
    return(rrt_merged_allpt_df)


rrt_merged_allpt_df= rrt_merging(rrtSQL_mv_ce_pt, rrtSQL_cv_pt, rrtSQL_mv_ie_pt, rrtSQL_mv_de_pt, rrtSQL_mv_pe_pt, timecol='charttime',time_var='first_pos_else_neg_ssc')

###RRTMERGED NOT WORKING

time: 71 ms


/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


In [39]:
# date= '27082018'

pd.DataFrame(rrt_merged_allpt_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_rrt_merged_allpt_df.csv' %(date))

time: 43.6 ms


## Urine Output

In [42]:
urine_output_all_df= clinvar_fxn(
    'urine_output_all',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/urine_output_all.sql' 
)

(75803, 5)
time: 10.1 s


# UTI related variables

In [43]:
uti_all_df= clinvar_fxn(
    'uti_all',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/uti_all.sql' 
)

(708156, 12)
time: 45.7 s


## Blood Gas Test

In [44]:
def PaO2(bg_all_nosummary_df):
    """
    overview: replaces the PO2 label with PaO2 on all instances (defined as sharing icustay_id and charttime being equal)
        where the specimen label == 'ART'
        
    input: bloodgas dataframe with values annotated. 
    output: bloodgas dataframe with values annotated where PO2 label is replaced with PaO2 according to above criteria
    """
    
    #making a unique varaible to search for and mark all rows where time and icustay_id has an art flag
    bg_all_nosummary_df['unique_var']= bg_all_nosummary_df['icustay_id'].map(str) +  bg_all_nosummary_df['charttime'].map(str)

    #making subset dataframe for label == SPECIMEN 
    bg_all_nosummary_specimen= bg_all_nosummary_df.loc[bg_all_nosummary_df.loc[:,'label']=='SPECIMEN',:]
    
    #all ART related rows: unique_var for all rows where label== SPECIMEN
    bg_all_nosummary_ART = bg_all_nosummary_specimen[bg_all_nosummary_specimen['value']=='ART'] 
    bg_all_nosummary_ART_list= list(bg_all_nosummary_ART['unique_var'].unique())

    #two criteria needed to change the PO2 to PaO2 label.
    criteria1=(bg_all_nosummary_df['label'] == 'PO2')
    criteria2=(bg_all_nosummary_df['unique_var'].isin(bg_all_nosummary_ART_list))
    
    #making changes
    bg_all_nosummary_df.loc[(criteria2 & criteria1),'label']= 'PaO2'
    
    return(bg_all_nosummary_df)

time: 14.5 ms


In [45]:
bg_all_nosummary_df= clinvar_fxn(
    'bg_all_nosummary',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/no_summary/bg_all_nosummary.sql' 
) 
bg_all_nosummary_df = PaO2(bg_all_nosummary_df)


(1664578, 8)
time: 1min 37s


In [46]:
bg_all_nosummary_df.head()

,subject_id,hadm_id,icustay_id,charttime,label,valuenum,value,valueuom,unique_var
607,9,150750,220597,2149-11-09 17:47:00,BASEEXCESS,1.0,1,mEq/L,2205972149-11-09 17:47:00
608,9,150750,220597,2149-11-09 17:47:00,CHLORIDE,103.0,103,mEq/L,2205972149-11-09 17:47:00
609,9,150750,220597,2149-11-09 17:47:00,FIO2,60.0,60,None,2205972149-11-09 17:47:00
610,9,150750,220597,2149-11-09 17:47:00,GLUCOSE,178.0,178,mg/dL,2205972149-11-09 17:47:00
611,9,150750,220597,2149-11-09 17:47:00,HEMATOCRIT,42.0,42,%,2205972149-11-09 17:47:00


time: 9.96 ms


## Vaso_active therapies

In [47]:
# 10/12/18 added amountuom as amount_uom, rateuom as rate_uom to many lines of the sql code.

weightdurations_df= clinvar_fxn(
    'weightdurations',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/weightdurations.sql'
)
#
epi_dose_df= clinvar_fxn(
    'epinephrine_dose',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/epinephrine_dose.sql'
)
#
norepi_dose_df= clinvar_fxn(
    'norepinephrine_dose',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/norepinephrine_dose.sql'
)
#
dopamine_dose_df= clinvar_fxn(
    'dopamine_dose',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/dopamine_dose.sql'
)
#
dobutamine_dose_df= clinvar_fxn(
    'dobutamine_dose',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/dobutamine_dose.sql'
)
#
vasopressin_dose_df= clinvar_fxn(
    'vasopressin_dose',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/vasopressin_dose.sql'
)

#removing units/hour, as these are not appropriate
vasopressin_dose_df= vasopressin_dose_df.loc[~vasopressin_dose_df.loc[:,'rate_uom'].isin(['Uhr','units/hour']),:]
#
phenylephrine_dose_df= clinvar_fxn(
    'phenylephrine_dose',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/phenylephrine_dose.sql'
)

(46991, 4)
(3637, 7)
(44522, 7)
(9941, 7)
(3199, 7)
(9280, 7)
(31665, 7)
time: 1min 1s


In [48]:
weightdurations_df= clinvar_fxn(
    'weightdurations',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/weightdurations.sql'
)

(46991, 4)
time: 4.28 s


In [49]:
#adding an identification label column and merging them into 1 df. 
epi_dose_df['label']='epinephrine'
norepi_dose_df['label']='norepinephrine'
dopamine_dose_df['label']='dopamine'
dobutamine_dose_df['label']='dobutamine'
vasopressin_dose_df['label']='vasopressin'
phenylephrine_dose_df['label']='phenylephrine'
vaso_dose_df = pd.concat([epi_dose_df, norepi_dose_df, dopamine_dose_df, dobutamine_dose_df, vasopressin_dose_df,phenylephrine_dose_df ])


time: 483 ms


# ventilator settings and categorization

In [50]:
#ventsettings_df = pd.read_csv('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/15082018_ventsettings_df.csv', index_col=0)
ventsettings_df= clinvar_fxn(
    'ventsettings',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/ventsettings.sql'
)
#going from wide format to long:
#pd.melt(ventsettings_df, id_vars=['icustay_id','charttime'])

(710323, 6)
time: 3min 56s


## rewritten vent script- more simple and generalizable

In [51]:
#ventsettings_df_3dayannotation
ventsettings_merge_df.head()

NameError: name 'ventsettings_merge_df' is not defined

time: 12.5 ms


In [52]:
def vent_data(vent_df,time_var='t_0', lower_window=0, upper_window=3 ):
    df= pd.merge(vent_df,
                final_pt_df2[['icustay_id',time_var]],
                left_on='icustay_id',
                right_on= 'icustay_id',
                how='left')
    df['delta']= pd.to_datetime(df['charttime']) - pd.to_datetime(df[time_var])
    df_timewindow= df.loc[df.loc[:,'delta']>= pd.Timedelta(days=lower_window),:]
    df_timewindow= df_timewindow.loc[df_timewindow.loc[:,'delta']<= pd.Timedelta(days=upper_window),:] 
    df_timewindow['day']= df_timewindow['delta'].apply(lambda x: pd.to_timedelta(x,unit='d').days) #day #
      
    return(df_timewindow)

#df_timewindow =vent_data(ventsettings_df,time_var='first_pos_else_neg_ssc', lower_window=-1, upper_window=1 )

def vent_day_categorizer(vent_df,time_var='t_0', lower_window=0, upper_window=3 ):
    
    df_timewindow =vent_data(vent_df,time_var=time_var, lower_window=lower_window, upper_window=upper_window)
    
    df_timewindow_perday=df_timewindow.groupby(['icustay_id','day'], as_index=False)[['mechvent','oxygentherapy']].agg({'mechvent':'max', 'oxygentherapy':'max'})

    conditions= [
        (df_timewindow_perday['mechvent']==1),
        ((df_timewindow_perday['oxygentherapy']==1) & (df_timewindow_perday['mechvent']==0)),
        (df_timewindow_perday['oxygentherapy']==0 & (df_timewindow_perday['mechvent']==0))]

    choices=['Mech', 'Oxygen', 'None']
    #
    df_timewindow_perday['value']= np.select(conditions, choices, default='error:no_value_filled')
    df_timewindow_perday['value']
    df_timewindow_perday=df_timewindow_perday.reset_index()
    df_timewindow_perday['uom']= 'mech/O2/none category'
    df_timewindow_perday= df_timewindow_perday.drop(['mechvent','oxygentherapy','index'], axis=1)
    df_timewindow_perday=pd.merge(df_timewindow_perday, final_pt_df2[['icustay_id',time_var]] )
    return(df_timewindow_perday)


time: 4.5 s


In [53]:
#ventcategory_df = vent_categorization(final_pt_df2, ventsettings_df, time_var='first_pos_else_neg_ssc' )
ventcategory_df= vent_day_categorizer(ventsettings_df,time_var='first_pos_else_neg_ssc', lower_window=lower_window, upper_window=upper_window)
#ventcount_df = vent_count(final_pt_df2,ventsettings_df, time_var='first_pos_else_neg_ssc')

time: 7.26 s


In [54]:
ventcategory_df.head()

,icustay_id,day,value,uom,first_pos_else_neg_ssc
0,200012.0,0,Oxygen,mech/O2/none category,2153-12-23 00:30:00
1,200014.0,0,Mech,mech/O2/none category,2105-02-17 00:00:00
2,200014.0,1,Oxygen,mech/O2/none category,2105-02-17 00:00:00
3,200019.0,0,Mech,mech/O2/none category,2178-07-08 21:51:00
4,200019.0,1,Oxygen,mech/O2/none category,2178-07-08 21:51:00


time: 14.3 ms


# daily SOFA score

### running yiheng's sql codes to capture daily sofa_score. ideally i coulda just used my data above but she had this written already so i'll use this.

link to her github: https://github.com/yihengpan/fluid_management/tree/master/sofa

###requirements for sofa_pan
sofa <- scorecalc <- scorecomp <- vaso_cv   
                                 <-wt      
                                 <-echo2

                   <- vaso_mv    
                   <- pafi2      <-bloodgas_pan_aterial <- bloodgas_pan  
                                 <-ventelurations
                   <- vitals_pan 
                   <- labs_pan   
                   <- uo_pan     
                   <- gcs_pan    
                                
                                

In [55]:
sofa_path= '/Users/geickelb1/Documents/GitHub/fluid_management/sofa'
var='wt'
wt_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

var='echo2'
echo2_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)


var='bloodgas_pan'
bloodgas_pan_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

var='bloodgas_pan_arterial'
bloodgas_pan_art_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

(22488, 3)
(14668, 2)
(188079, 33)
(152414, 39)
time: 3min 12s


In [58]:
#vaso_mv, vaso_cv,pafi2, vitals_pan, labs_pan, uo_pan, gcs_pan
sofa_path= '/Users/geickelb1/Documents/GitHub/fluid_management/sofa'
var='vaso_mv'
vaso_mv_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

var='vaso_mv'
vaso_mv_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

var='vaso_cv'
vaso_cv_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

var='pafi1'
pafi1_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

var='pafi2'
pafi2_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)


var='labs_pan'
labs_pan_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

var='uo_pan'
uo_pan_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

var='gcs_pan'
gcs_pan_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

###

var='scorecomp'
scorecomp_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

var='scorecalc'
scorecalc_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

var='sofa_pan'
sofa_pan_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

(5732, 6)
(5732, 6)
(6883, 6)
(152414, 5)
(48461, 4)
(76926, 42)
(75721, 5)
(14486, 9)
(87899, 14)
(87899, 20)
(83629, 23)
time: 8min 7s


In [60]:
#deleting these to clear up memory 

del(vaso_cv_df, vaso_mv_df, labs_pan_df, gcs_pan_df, scorecalc_df, scorecomp_df, uo_pan_df, pafi1_df, pafi2_df, bloodgas_pan_art_df, echo2_df, wt_df)

time: 28 ms


In [61]:
sofa_pan_df['hadm_id'].nunique() #8707
final_pt_df2['hadm_id'].nunique() #8731

14668

time: 5.59 ms


In [62]:
#adding in t_0 & icuadmit date
def sofa_day_window_filter(sofa_pan_df, time_var= 't_0', lower_window= 0, upper_window=3): #'t_0'):
    import datetime
    '''
    #Yihangpan wrote a sql script and materialized view "sofa_pan" which gives the sofa score for each day in icu for each patient. 
    #since the sofa_pan has days after admission but not chartdates, I need to use day # to find the associated t_0 to t_0+72 for each patient. 
    # the challenge was that I had to relate day# in sofa_pan to my t_0 date. the day # was based on the days after icu admission, where day1 = the first day (0 to 24 hours post admission). this was changed so day 0= 0 to 24 hours.
    #To do this, I added day# (where day 0 is the first day) to icu admission date. 
    #I then filtered on only the rows where this icuadmin + day# was between t_0 and t_0 + 72 hours. 

    #since t_0 has only day resolution, and for that I ignored hours and only took the date (rounded down all hours/minutes/seconds). this is similar to how i made the t_0 date. 
    #the problem this creates is that it widens the potential time window, so it theoretically can contain up to 95.99 hours, since hours on day 1 were collapsed to 0. 

    
    input: 
        sofa_pan_df: daily sofa scores captured from sofa_pan_sql.
        optional:
            time_var: the time variable we want to base the window off of
            window_bottom= 0, time_var- window_bottom (days + time_var) = first daily sofa score to capture
            window_top= 0, time_var- window_top (days + time_var) = last daily sofa score to capture 
    
    output: sofa_pan_sql annotated with days and filtered to time window set by window_bottom and window_top. 
    '''
    time_var= 'first_pos_else_neg_ssc' 

    ##merging sofa_pan with final_pt df in order to append on icustay_id, the time var, and ICU_admit
    sofa_df_merged= pd.merge(sofa_pan_df,
                             final_pt_df2[['icustay_id',time_var,'ICU_admit']],
                             left_on= 'icustay_id',
                             right_on = 'icustay_id',
                             how='left') #n=240317

    #sofa_df_merged['admit_t0_rounded'] = pd.to_datetime(sofa_df_merged['ICU_admit']).dt.round('1440min')
    
    sofa_df_merged['day'] = sofa_df_merged['day']-1 #putting the epoch days so that 0 = the first day
    
    #approximating the charttime of the time associated with each sofa score
    sofa_df_merged['approx_charttime']=pd.to_timedelta((sofa_df_merged['day'])*24, unit='h') + pd.to_datetime(sofa_df_merged['ICU_admit'])

    #rounding down the charttime to the day, so hours and minutes are ignored (just like t_0)
    sofa_df_merged['floor_charttime'] = sofa_df_merged['approx_charttime'].apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day, 24*(dt.hour//24))) 
    
    sofa_df_merged['floor_time_var'] = pd.to_datetime(sofa_df_merged[time_var]).apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day, 24*(dt.hour//24))) 
    
    sofa_df_72= sofa_df_merged.loc[
        (sofa_df_merged['floor_charttime'].between(
            (pd.to_datetime(sofa_df_merged['floor_time_var'])+ pd.to_timedelta(lower_window, unit='d')),
            (pd.to_datetime(sofa_df_merged['floor_time_var'])+ pd.to_timedelta(upper_window, unit='d')+ pd.to_timedelta(1, unit='h')) #added 1hr timebuffer incase between is set as less than greater than
        )),:]
    
    return(sofa_df_72.drop(['floor_time_var','floor_charttime'], axis=1))


time: 30.4 ms


In [64]:
sofa_df_72= sofa_day_window_filter(sofa_pan_df, time_var= 'first_pos_else_neg_ssc', lower_window= lower_window, upper_window=upper_window)

time: 1.13 s


In [66]:
sofa_df_72.head()

,subject_id,hadm_id,icustay_id,day,sofa,respiration,pao2fio2_vent_min,pao2fio2_novent_min,coagulation,platelet_min,...,rate_dobutamine,meanbp_min,cns,mingcs,renal,creatinine_max,urineoutput,first_pos_else_neg_ssc,ICU_admit,approx_charttime
4,9,150750,220597,4.0,6,0.0,204.0,NaN,0.0,306.0,...,NaN,54.0,NaN,NaN,2.0,2.0,6020.0,2149-11-13 19:30:00,2149-11-09 13:07:02,2149-11-13 13:07:02
5,9,150750,220597,5.0,7,NaN,NaN,NaN,NaN,NaN,...,NaN,91.0,NaN,NaN,4.0,NaN,100.0,2149-11-13 19:30:00,2149-11-09 13:07:02,2149-11-14 13:07:02
6,19,109235,273430,0.0,2,NaN,NaN,NaN,0.0,216.0,...,NaN,75.0,1.0,14.0,1.0,1.2,940.0,2108-08-05 20:42:00,2108-08-05 16:26:09,2108-08-05 16:26:09
7,19,109235,273430,1.0,0,NaN,NaN,NaN,NaN,NaN,...,NaN,78.0,NaN,NaN,0.0,NaN,780.0,2108-08-05 20:42:00,2108-08-05 16:26:09,2108-08-06 16:26:09
8,25,129635,203487,0.0,4,NaN,NaN,NaN,0.0,172.0,...,NaN,24.0,NaN,NaN,1.0,1.6,3025.0,2160-11-02 08:51:00,2160-11-02 03:16:23,2160-11-02 03:16:23


time: 19.5 ms


In [79]:
vitals_all_nosummary_df

,subject_id,hadm_id,icustay_id,charttime,valueuom,vitalid,valuenum
3,9,150750,220597,2149-11-11 00:00:00,mmHg,MeanArtPress,81.000000
4,9,150750,220597,2149-11-11 19:00:00,mmHg,MeanArtPress,72.000000
5,9,150750,220597,2149-11-11 19:45:00,mmHg,MeanArtPress,76.000000
6,9,150750,220597,2149-11-11 20:00:00,mmHg,MeanArtPress,99.000000
17,25,129635,203487,2160-11-02 15:00:00,mmHg,MeanArtPress,82.000000
18,25,129635,203487,2160-11-02 16:00:00,mmHg,MeanArtPress,92.000000
19,25,129635,203487,2160-11-02 17:00:00,mmHg,MeanArtPress,71.000000
20,9,150750,220597,2149-11-09 16:00:00,mmHg,MeanArtPress,105.000000
25,9,150750,220597,2149-11-13 15:20:00,mmHg,MeanArtPress,61.000000
26,9,150750,220597,2149-11-14 11:00:00,mmHg,MeanArtPress,103.000000


time: 43.3 ms


# saving as csv

In [82]:
#####NEED TO CLEAN UP PATHWAY CODING


pd.DataFrame(vaso_dose_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_vaso_dose_df.csv' %(date))
pd.DataFrame(ventsettings_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_ventsettings_df.csv' %(date))
# pd.DataFrame(ventcount_df).to_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_ventcount_df.csv' %(date)) #not useful

# pd.DataFrame(echodata_df).to_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_echodata_df.csv' %(date))
pd.DataFrame(weightfirstday_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_weightfirstday_df.csv' %(date))
pd.DataFrame(heightfirstday_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_heightfirstday_df.csv' %(date))
pd.DataFrame(labs_all_nosummary_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_labs_all_nosummary_df.csv' %(date))

pd.DataFrame(vitals_all_nosummary_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_vitals_all_nosummary_df.csv' %(date))

# pd.DataFrame(gcsall_df).to_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_gcsall_df.csv' %(date))

pd.DataFrame(urine_output_all_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_urine_output_all_df.csv' %(date))
pd.DataFrame(uti_all_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_uti_all_df.csv' %(date))
pd.DataFrame(bg_all_nosummary_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_bg_all_nosummary_df.csv' %(date))

#timewindowed
pd.DataFrame(ventcategory_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_ventcategory_df.csv' %(date))
pd.DataFrame(rrt_merged_allpt_df).to_csv( 
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_rrt_all_df.csv' %(date)) #timewindowed
pd.DataFrame(gcs72_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_gcs72_df.csv' %(date)) #gcs within time window here
pd.DataFrame(sofa_df_72).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_sofa_df_72.csv' %(date))

time: 2min 43s


# time window filtering 8/28/18  (updated 1/28/19)
saving a new version of each clincal variable dataframe that is filtered to only 72 hour window after t_0 for each icustay_id 

- need to organize this better 

### reading in data if needed

In [256]:
# #final_pt_df2 = pd.read_csv('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/16082018_final_pt_df2.csv', index_col=0)

# #large import of all data 

# date= '27082018'

# vaso_dose_df =pd.read_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_vaso_dose_df.csv' %(date), index_col=0)
# ventsettings_df=pd.read_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_ventsettings_df.csv' %(date), index_col=0)
# ventcategory_df=pd.read_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_ventcategory_df.csv' %(date), index_col=0)
# ventcount_df=pd.read_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_ventcount_df.csv' %(date), index_col=0)

# echodata_df=pd.read_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_echodata_df.csv' %(date), index_col=0)
# weightfirstday_df=pd.read_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_weightfirstday_df.csv' %(date), index_col=0)
# heightfirstday_df=pd.read_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_heightfirstday_df.csv' %(date), index_col=0)
# labs_all_nosummary_df=pd.read_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_labs_all_nosummary_df.csv' %(date), index_col=0)
# vitals_all_nosummary_df=pd.read_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_vitals_all_nosummary_df.csv' %(date), index_col=0)
# gcsall_df=pd.read_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_gcsall_df.csv' %(date), index_col=0)
# rrt_all_df=pd.read_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_rrt_all_df.csv' %(date), index_col=0)
# urine_output_all_df=pd.read_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_urine_output_all_df.csv' %(date), index_col=0)
# uti_all_df=pd.read_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_uti_all_df.csv' %(date), index_col=0)
# bg_all_nosummary_df=pd.read_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_bg_all_nosummary_df.csv' %(date), index_col=0)


/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


time: 32.6 s


In [69]:
def time_window_filter(df, final_pt_df2,timecol,time_var='t_0', lower_window=0, upper_window=3):
    try:
        df= pd.merge(df, final_pt_df2[['icustay_id',time_var]], left_on= 'icustay_id', right_on = 'icustay_id') #n=240317
        df['delta']= pd.to_datetime(df[timecol]) - pd.to_datetime(df[time_var])
        df_after_t0= df.loc[df.loc[:,'delta']>= pd.Timedelta(days=lower_window),:]
        df_after_t0= df_after_t0.loc[df_after_t0.loc[:,'delta']<= pd.Timedelta(days=upper_window),:]
    except KeyError or NameError:
        df= pd.merge(df, final_pt_df2[['hadm_id',time_var]], left_on= 'hadm_id', right_on = 'hadm_id') #n=240317
        df['delta']= pd.to_datetime(df[timecol]) - pd.to_datetime(df[time_var])
        df_after_t0= df.loc[df.loc[:,'delta']>= pd.Timedelta(days=lower_window),:]
        df_after_t0= df_after_t0.loc[df_after_t0.loc[:,'delta']<= pd.Timedelta(days=upper_window),:]
    return(df_after_t0)

time: 18.8 ms


In [81]:
###list of data w/ 72 hour adjustments
#vaso_dose_df - vaso_dose_72 -
#ventsettings_df - ventsettings_72 -
#ventcategory_df - ventcategory_df - 
#echodata_df - echodata_72 -
#labs_all_nosummary_df- labs_all_nosummary_72 -
#vitals_all_nosummary_df - vitals_all_nosummary_72 -
#gcsall_df - Gcs72_df **modified the gcs in python above. within 72 hour window. -
#rrt_all_df - rrt_merged_allpt_df *modified the rrt in python above. within 72 hour window.  -
#uti_all_df - uti_all_72 -
#bg_all_nosummary_df - bg_all_nosummary_72
#sofa_df_72 #already within 72hour window

#urine_output_all_df - . xxx this also doesn't have times, but this won't be used much so i didn't bother editing.

vaso_dose_72= time_window_filter(vaso_dose_df, final_pt_df2, 'starttime',time_var='first_pos_else_neg_ssc', lower_window=lower_window, upper_window=upper_window)
ventsettings_72= time_window_filter(ventsettings_df, final_pt_df2, "charttime",time_var='first_pos_else_neg_ssc', lower_window=lower_window, upper_window=upper_window)
# echodata_72= time_window_filter(echodata_df, final_pt_df2, 'charttime',time_var='first_pos_else_neg_ssc', lower_window=lower_window, upper_window=upper_window)
labs_all_nosummary_72= time_window_filter(labs_all_nosummary_df, final_pt_df2, "charttime",time_var='first_pos_else_neg_ssc', lower_window=lower_window, upper_window=upper_window)
vitals_all_nosummary_72 = time_window_filter(vitals_all_nosummary_df, final_pt_df2, "charttime",time_var='first_pos_else_neg_ssc', lower_window=lower_window, upper_window=upper_window)
uti_all_72 = time_window_filter(uti_all_df, final_pt_df2, 'charttime',time_var='first_pos_else_neg_ssc', lower_window=lower_window, upper_window=upper_window)
bg_all_nosummary_72 = time_window_filter(bg_all_nosummary_df, final_pt_df2, 'charttime',time_var='first_pos_else_neg_ssc', lower_window=lower_window, upper_window=upper_window)

time: 44.1 s


# filtering to patients with bare minimum vital numbers
06.1-QC_and_missingness found that 3% or so of patients don't have baseline vitals counts. this is filtering the patients to only those who have this baseline value.

as of 10/12/18, this code has not been implemented in here and is performed after importing. (updated below to be self contained in 1/28/19)

In [11]:
# date= '25012019'

# vitals_all_nosummary_df =pd.read_csv(
#      Path('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_vitals_all_nosummary_df.csv' %(date)), index_col=0)


/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


time: 13.7 s


In [83]:
#using origional criteria to find pts who have atleast 1 spo2 reading within 3 days of t_0

#The idea is that this should be the bare minimum amount of data for a patient, and without it, it's likely the physicians did not suspect an infection in these patients.

vitals_filter = time_window_filter(vitals_all_nosummary_df, final_pt_df2, "charttime",time_var='t_0', lower_window=-1, upper_window=3)

vitals_filter= vitals_filter.loc[
    vitals_filter['vitalid'].notnull(),:]#.count() #6930 NULL values

icustay_id_vitals = (vitals_filter.loc[
    vitals_filter.loc[:,'vitalid']=='SpO2','icustay_id'
        ].unique())

len(icustay_id) #8731
len(icustay_id_vitals) #8629
subject_id_vitals=list(final_pt_df2.loc[final_pt_df2.loc[:,'icustay_id'].isin(icustay_id_vitals),'subject_id'])
hadm_id_vitals= list(final_pt_df2.loc[final_pt_df2.loc[:,'icustay_id'].isin(icustay_id_vitals),'hadm_id'])
icustay_id_vitals= list(final_pt_df2.loc[final_pt_df2.loc[:,'icustay_id'].isin(icustay_id_vitals),'icustay_id'])
del(vitals_filter)

time: 20.7 s


In [23]:
# vitals_all_nosummary_72= df_subset(vitals_all_nosummary_72)
# pd.DataFrame(vitals_all_nosummary_72).to_csv("/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_vitals_all_nosummary_72.csv" %date) #final cohort database n=11493 subject_id’s (7/6/18)


time: 46 s


In [84]:
#saving this slightly reduced cohort for those who have sufficient vitals
# date= '25012019'
final_pt_df2_v=final_pt_df2.loc[final_pt_df2.loc[:,'icustay_id'].isin(icustay_id_vitals),:]
pd.DataFrame(final_pt_df2_v).to_csv("/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_final_pt_df2_v.csv" %date) #final cohort database n=11493 subject_id’s (7/6/18)


time: 301 ms


In [85]:
def df_subset(df):
    try:
        df = df.loc[df.loc[:,'icustay_id'].isin(icustay_id_vitals),:]
    except KeyError or NameError:
        try:
            df = df.loc[df.loc[:,'hadm_id'].isin(hadm_id_vitals),:]
        except KeyError or NameError:
            df = df.loc[df.loc[:,'subject_id'].isin(subject_id_vitals),:]
    return(df)

time: 16.7 ms


In [111]:
# subsetting each dataframe to only patients in final_patients_df2_v:
dataframe_list= [
    age_df, ventcategory_df, vaso_dose_72, #echodata_72,
    labs_all_nosummary_72, weightfirstday_df,
    heightfirstday_df, vitals_all_nosummary_72,
    uti_all_72, bg_all_nosummary_72,
    rrt_merged_allpt_df, gcs72_df, sofa_df_72
]

(
age_df, ventcategory_df, vaso_dose_72, #echodata_72,
labs_all_nosummary_72, weightfirstday_df,
heightfirstday_df, vitals_all_nosummary_72,
uti_all_72, bg_all_nosummary_72,
rrt_merged_allpt_df, gcs72_df, sofa_df_72
) = (df_subset(df) for df in dataframe_list)

time: 9.26 s


In [140]:
bg_all_nosummary_72

,subject_id,hadm_id,icustay_id,charttime,label,valuenum,value,valueuom,unique_var,first_pos_else_neg_ssc,delta
110,9,150750,220597,2149-11-14 01:44:00,BASEEXCESS,NaN,0,mEq/L,2205972149-11-14 01:44:00,2149-11-13 19:30:00,0 days 06:14:00
111,9,150750,220597,2149-11-14 01:44:00,FIO2,50.00,50,None,2205972149-11-14 01:44:00,2149-11-13 19:30:00,0 days 06:14:00
112,9,150750,220597,2149-11-14 01:44:00,INTUBATED,NaN,INTUBATED,None,2205972149-11-14 01:44:00,2149-11-13 19:30:00,0 days 06:14:00
113,9,150750,220597,2149-11-14 01:44:00,PCO2,23.00,23,mm Hg,2205972149-11-14 01:44:00,2149-11-13 19:30:00,0 days 06:14:00
114,9,150750,220597,2149-11-14 01:44:00,PEEP,5.00,5,None,2205972149-11-14 01:44:00,2149-11-13 19:30:00,0 days 06:14:00
115,9,150750,220597,2149-11-14 01:44:00,PH,7.57,7.57,units,2205972149-11-14 01:44:00,2149-11-13 19:30:00,0 days 06:14:00
116,9,150750,220597,2149-11-14 01:44:00,PaO2,102.00,102,mm Hg,2205972149-11-14 01:44:00,2149-11-13 19:30:00,0 days 06:14:00
117,9,150750,220597,2149-11-14 01:44:00,SPECIMEN,NaN,ART,None,2205972149-11-14 01:44:00,2149-11-13 19:30:00,0 days 06:14:00
118,9,150750,220597,2149-11-14 01:44:00,TEMPERATURE,35.60,35.6,None,2205972149-11-14 01:44:00,2149-11-13 19:30:00,0 days 06:14:00
119,9,150750,220597,2149-11-14 01:44:00,TIDALVOLUME,700.00,700,None,2205972149-11-14 01:44:00,2149-11-13 19:30:00,0 days 06:14:00


time: 202 ms


In [95]:
#date= '27082018'
# date= '25012019'

pd.DataFrame(age_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/72_hr_window/%s_pt_info_df.csv' %(date))
pd.DataFrame(vaso_dose_72).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/72_hr_window/%s_vaso_dose_72.csv' %(date))

pd.DataFrame(vaso_dose_72).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/72_hr_window/%s_vaso_dose_72.csv' %(date))
pd.DataFrame(ventsettings_72).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/72_hr_window/%s_ventsettings_72.csv' %(date))

#vent category and count are already 72hour
pd.DataFrame(ventsettings_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/72_hr_window/%s_ventsettings_df.csv' %(date))
pd.DataFrame(ventcategory_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/72_hr_window/%s_ventcategory_df.csv' %(date))

# pd.DataFrame(echodata_72).to_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/72_hr_window/%s_echodata_72.csv' %(date))
pd.DataFrame(labs_all_nosummary_72).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/72_hr_window/%s_labs_all_nosummary_72.csv' %(date))

#vent category and count are already limited to first day
pd.DataFrame(weightfirstday_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/72_hr_window/%s_weightfirstday_df.csv' %(date))
pd.DataFrame(heightfirstday_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/72_hr_window/%s_heightfirstday_df.csv' %(date))

pd.DataFrame(vitals_all_nosummary_72).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/72_hr_window/%s_vitals_all_nosummary_72.csv' %(date))
pd.DataFrame(uti_all_72).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/72_hr_window/%s_uti_all_72.csv' %(date))
pd.DataFrame(bg_all_nosummary_72).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/72_hr_window/%s_bg_all_nosummary_72.csv' %(date))

pd.DataFrame(rrt_merged_allpt_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/72_hr_window/%s_rrt_merged_allpt_df.csv' %(date))
pd.DataFrame(gcs72_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/72_hr_window/%s_gcs72_df.csv' %(date))

pd.DataFrame(sofa_df_72).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/72_hr_window/%s_sofa_df_72.csv' %(date))


time: 3min 9s


'04042019'

time: 7.78 ms
